In [1]:
import gurobipy as gp
from gurobipy import GRB
import math

In [2]:
def calculate_optimal_gurobi(l0=None, l1=None, l2=None, B=None, lb=None, ub=None, M=100):
    m = gp.Model('box_constraint_test')
    m.setParam( 'OutputFlag', False )
    # Variables
    
    Bi = m.addVar(lb=lb, ub=ub, vtype=GRB.CONTINUOUS, name="Bi")
    Bi_supp = m.addVar(vtype=GRB.BINARY, name = '1[Bi]')
    Bi_sign = m.addVar(vtype=GRB.BINARY)
    Bi_pos = m.addVar(lb=0, ub=ub, vtype=GRB.CONTINUOUS)
    Bi_neg = m.addVar(lb=0, ub=-lb, vtype=GRB.CONTINUOUS)
    
    #Constraints
    m.addConstr(Bi_pos <= M*Bi_sign)
    m.addConstr(Bi_neg <= M*(1-Bi_sign))
    m.addConstr(Bi == Bi_pos - Bi_neg)

    m.addConstr(Bi >= -M*Bi_supp)
    m.addConstr(Bi <= M*Bi_supp)
    
    #Cost
    m.setObjective(0.5*(B - Bi)*(B - Bi) 
                   + l0*Bi_supp
                   + l1*(Bi_pos + Bi_neg) 
                   + l2*Bi*Bi, GRB.MINIMIZE)
    
    m.optimize()
    
    for v in m.getVars():
        if v.varName == "Bi":
            return v.x

In [56]:
math.copysign?

Signature: math.copysign(x, y, /)
Docstring:
Return a float with the magnitude (absolute value) of x but the sign of y.

On platforms that support signed zeros, copysign(1.0, -0.0)
returns -1.0.
Type:      builtin_function_or_method


In [511]:
def box(x, lb, ub):
    if x > ub:
        return ub
    elif x <= lb:
        return lb
    else:
        return x
    
def sign(x):
    if x < 0:
        return -1
    return 1


def calculate_close_form2(l0=None, l1=None, l2=None, B=None, lb=None, ub=None, M=None):
    Bi = B
    
    two_l2_p1 = 1 + 2*l2
    x_opt_no_L0 = math.copysign((Bi - l1)/two_l2_p1, Bi)
    
    if abs(x_opt_no_L0) <= math.sqrt(2*l0/two_l2_p1):
        # x_opt without bounds does not improve cost by more than the penalties/
        # Set x to 0.
        return 0
        
    x = box(x_opt_no_L0, lb, ub)
    # Box x to normal size.
    
    
    delta = math.sqrt(((abs(Bi) - l1)**2 - 2*l0*two_l2_p1)/two_l2_p1)
    x_range_low = x_opt_no_L0 - delta
    x_range_high = x_opt_no_L0 + delta
    
    print("Clamped x is: ", x)
    print("x magnitude needed: ", math.sqrt(2*l0/two_l2_p1))
    print(f"x range needed: [{x_range_low}, {x_range_high}]")
    
    return x
    if x_range_low <= x <= x_range_high:
        return 0
    else:
        return x
    

def calculate_closed_form(l0=None, l1=None, l2=None, B=None, lb=None, ub=None, M=None):
    Bi = B # Attempt to make Bi, B. This is solved for iteratively using other approaches
    
    two_l2_p1 = 1 + 2*l2
    
    x_opt_no_L0 = math.copysign((Bi - l1)/two_l2_p1, Bi)
    print("x magnitude needed: ", math.sqrt(2*l0/two_l2_p1))
    
    x = box(x_opt_no_L0, lb, ub)
    
    delta = math.sqrt(((abs(Bi) - l1)**2 - 2*l0*two_l2_p1)/two_l2_p1)
     
    x_range_low = x_opt_no_L0 - delta
    x_range_high = x_opt_no_L0 + delta
    
    #print(x_range_low, x_range_high)
    
    print("Clamped x is: ", x)
    print("x magnitude needed: ", math.sqrt(2*l0/two_l2_p1))
    print(f"x range needed: [{x_range_low}, {x_range_high}]")
    
    return x*(abs(x_opt_no_L0) >= math.sqrt(2*l0/two_l2_p1) and x_range_low <= x <= x_range_high)


def calculate_closed_form3(l0=None, l1=None, l2=None, B=None, lb=None, ub=None, M=None):
    Bi = B # Attempt to make Bi, B. This is solved for iteratively using other approaches
    
    if l1 >= abs(Bi):
        return 0
    
    print((abs(Bi) - l0)/(1 + 2*l2))
    if (abs(Bi) - l0)/(1 + 2*l2) < math.sqrt(2*l0/(1 + 2*l2)):
        return 0
    
    elif (abs(Bi) - l0)/(1 + 2*l2) >= math.sqrt(2*l0/(1 + 2*l2)):
        
        delta = math.sqrt(((abs(Bi) - l1)**2 - 2*l0*(1 + 2*l2)))
        low = sign(Bi)*(abs(Bi) - l1)/(1+2*l2) - delta
        high = sign(Bi)*(abs(Bi) - l1)/(1+2*l2) + delta
        x = box(sign(Bi)*(abs(Bi) - l1)/(1+2*l2), lb, ub)
        print("x box")
        print(f"range: [{low}, {high}]")
        if low <= x <= high:
            return x
        else:
            return 0

In [546]:
def cost(B, Bi, l0, l1, l2):
    return 1/2*(B - Bi)**2 + l0*(Bi!=0) + l1*abs(Bi) + l2*Bi**2

In [550]:
l0=0.006
l1=0.0
l2=1e-4
B=4
lb=0.0
ub=0.001

In [551]:
Bi_optimal = calculate_optimal_gurobi(l0=l0, l1=l1, l2=l2, B=B , lb=lb, ub=ub, M = 1000)
print(Bi_optimal, cost(B, Bi_optimal, l0, l1, l2))

0.0 8.0


In [552]:
Bi_box = calculate_closed_form3(l0=l0, l1=l1, l2=l2, B=B , lb=lb, ub=ub)
print(Bi_box, cost(B, Bi_box, l0, l1, l2))

3.993201359728055
x box
range: [0.000700741436099328, 7.997699578499914]
0.001 8.0020005001


Test the implementations with 'hypothesis'. <br>
hypothesis will search through the space defined by @given(...) and try to find examples that fail. It will then search the "neighbors" of that example trying to make as many of the variables 0 as possible.

In [437]:
import hypothesis
from hypothesis import given, settings, assume, example
from hypothesis.strategies import floats
import numpy as np

For example:
```python
def add(x, y):
    return x - y

@given(x=floats(0, 10), y=floats(0, 10))
def test_add(x, y):
    # x and y will be assigned floating values between 0, 10
    assert add(x, y) == x + y
    
test_add() # Run here
```

test_add will try different values of x and y until it finds a failure. It will then try to push x and y to zero to find the "smallest" error. It will successfully push 'x' to zero as the 'truthyness" of x-y == x+y does not depend on x. It only depends on y. Therefore, the failure it will find should be x=0 and y being very small.

In [438]:
def add(x, y):
    return x - y

@given(x=floats(0, 10), y=floats(0, 10))
def test_add(x, y):
    # x and y will be assigned floating values between 0, 10
    assert add(x, y) == x + y
    
test_add()

Falsifying example: test_add(
    x=0.0, y=2.2204460492503135e-15,
)


AssertionError: 

# Lets run on our problem

# Compare values

In [458]:
@given(l0=floats(0, 10), l1 = floats(0, 10), l2 = floats(0, 10),
       B=floats(-10, 10), lb=floats(-1, 0), ub=floats(0, 1))
@settings(max_examples=1000)
@example(l0=0, l1=0, l2=0, B=1, lb=-1, ub=1)
@example(l0=0, l1=0, l2=10, B=1, lb=-1, ub=1)
@example(l0=0, l1=10, l2=0, B=1, lb=-1, ub=1)
@example(l0=0, l1=10, l2=10, B=1, lb=-1, ub=1)
@example(l0=10, l1=0, l2=10, B=1, lb=-1, ub=1)
@example(l0=10, l1=10, l2=0, B=1, lb=-1, ub=1)
@example(l0=10, l1=10, l2=10, B=1, lb=-1, ub=1)
@example(l0=0, l1=0, l2 =0,B =-0.1, lb=-0.02, ub=0)
@example(l0=0.0061658496635019775, l1=0.0, l2=1e-4, B=4, lb=0.0, ub=0.0015000000000000573)
def compare_B(l0, l1, l2, B, lb, ub):
    try:
        np.testing.assert_almost_equal(calculate_optimal_gurobi(l0=l0, l1=l1, l2=l2, B=B , lb=lb, ub=ub),
                                       calculate_closed_form3(l0=l0, l1=l1, l2=l2, B=B , lb=lb, ub=ub), decimal=3)
    except ValueError:
        pass

In [459]:
#

In [510]:
#compare_B()

In [507]:
l0=0.0061658496635019775
l1=0.0
l2=1e-4
B=4
lb=0.0
ub=0.0015000000000000573

In [508]:
Bi_box = calculate_closed_form3(l0=l0, l1=l1, l2=l2, B=B , lb=lb, ub=ub)
print(Bi_box, cost(B, Bi_box, l0, l1, l2))

3.9930355432278524
x box
0.0015000000000000573 15.994168099888501


In [509]:
Bi_optimal = calculate_optimal_gurobi(l0=l0, l1=l1, l2=l2, B=B , lb=lb, ub=ub)
print(Bi_optimal, cost(B, Bi_optimal, l0, l1, l2))

0.0 16.0


# Compare Objectives

In [21]:
@given(l0=floats(0, 10), l1 = floats(0, 10), l2 = floats(0, 10),
       B=floats(0, 10), lb=floats(-1, 0), ub=floats(0, 1))
@settings(max_examples=1000, derandomize=True)
def compare_obj(l0, l1, l2, B, lb, ub):
    global decimal
    Bi_optimal = calculate_optimal_gurobi(l0=l0, l1=l1, l2=l2, B=B , lb=lb, ub=ub)
    Bi_box = calculate_closed_form(l0=l0, l1=l1, l2=l2, B=B , lb=lb, ub=ub)
    
    try:
        np.testing.assert_almost_equal(cost(B, Bi_optimal, l0, l1, l2),
                                       cost(B, Bi_box, l0, l1, l2), decimal=decimal)
    except AssertionError as e:
        print(Bi_optimal, Bi_box)
        raise 

In [23]:
decimal = 6
compare_obj()